# 基于SD的特征追踪

## 这个示例展示一个在AOP上运行的特征点追踪算法

调用接口：
- 用opencv的sift
- tianmoucv.proc.tracking.Feature_tracker_sd

In [ ]:
%load_ext autoreload

## 必要的包

In [ ]:
%autoreload

import argparse
import os
import sys
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
import math

In [ ]:
%autoreload
import time
from tianmoucv.isp import SD2XY
from tianmoucv.proc.features import HarrisCorner,steadyHarrisCornerForSIFT
from tianmoucv.proc.tracking import Feature_tracker_sd
from tianmoucv.proc.reconstruct import poisson_blending
from tianmoucv.data import TianmoucDataReader

from IPython.display import clear_output
import cv2

gap  = 5
#目前的策略比较粗暴，间隔一个固定的帧就直接重新找特征点，防止积累误差
fix_update_frame = 250
time_begin = time.time()

all_data='/data/lyh/tianmoucData/20240930_tobi_sup_exp/data/QRCode_compare/tianmouc/not_known/'

d_factor = 0.75

for key in ['1000r']:
    
    dataset = TianmoucDataReader(all_data,MAXLEN=500,matchkey=key)
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1,\
                                          num_workers=4, pin_memory=False, drop_last = False)
    imlist = []
    tracking_count = 0
    traker_bilinear = Feature_tracker_sd(d_factor=d_factor)

    for index,sample in enumerate(dataLoader,0):
        if index >5:
            break

        tsdiff_160x160_b = sample['rawDiff'][0,...]/255.0
        tsdiff = sample['tsdiff'][0,...]/255.0
        TD_160x320_acc = 0
        TD_nn_acc = 0
        SD_160x320_acc = 0
        SD_nn_acc = 0
        
        for t in range(tsdiff_160x160_b.size(1)):
            tsdiff_160x160 = tsdiff_160x160_b[:,t,...]
            SD = tsdiff_160x160[1:,...].permute(1,2,0)
            Ix,Iy= SD2XY(SD)

            sobel = (Ix**2+Iy**2)**0.5
            history, imshow_pos = traker_bilinear.update(sobel,viz=True)
            imlist.append(imshow_pos)

            if t % 12 == 0:
                time_10 = time.time()
                clear_output()
                plt.subplot(1,1,1)  
                print('avg fps:',10/(time_10-time_begin))
                time_begin =time.time()
                plt.imshow(imshow_pos,cmap='gray')
                plt.show()

In [ ]:
def images_to_video(frame_list,name,size=(640,320),Flip=True):
    fps = 30        
    ftmax = 1
    ftmin = 0
    print(frame_list[0].shape,len(frame_list))
    size = (frame_list[0].shape[1],frame_list[0].shape[0])
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for ft in frame_list:
        ft2 = (ft).astype(np.uint8)
        out.write(ft2)
    out.release()

images_to_video(imlist,'./feature_tracking.mp4',Flip=False)